<a href="https://colab.research.google.com/github/akash007data/akashl-learning-git/blob/main/Assignment2_G24AI2040_AKASH_TIWARI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**link for colab**: https://colab.research.google.com/drive/1iMBUbLZvM3KxCMb7xM0nEPIeYIdk8qCk?usp=sharing

In [ ]:
# downloading the csv files directly from assignment link

# cruise ships data
!wget https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/cruise.csv

# customer churn data
!wget https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/customer_churn.csv

# e-commerce customer data
!wget https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/e-com_customer.csv


--2025-07-29 10:09:58--  https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/cruise.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8734 (8.5K) [text/plain]
Saving to: ‘cruise.csv’

cruise.csv          100%[===================>]   8.53K  --.-KB/s    in 0s      

2025-07-29 10:09:59 (52.9 MB/s) - ‘cruise.csv’ saved [8734/8734]

--2025-07-29 10:09:59--  https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/customer_churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response

In [ ]:
# Install Java (OpenJDK 11) for Hadoop
!sudo apt-get update -y
!sudo apt-get install openjdk-11-jdk-headless -y


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
#download Hadoop
!wget https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz

--2025-07-29 10:03:04--  https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.208.237, 135.181.214.104, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.208.237|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 730107476 (696M) [application/x-gzip]
Saving to: ‘hadoop-3.3.6.tar.gz’

hadoop-3.3.6.tar.gz 100%[===================>] 696.28M  20.0MB/s    in 38s     

2025-07-29 10:03:43 (18.2 MB/s) - ‘hadoop-3.3.6.tar.gz’ saved [730107476/730107476]



In [ ]:
#Extract and move Hadoop
!tar -xzf hadoop-3.3.6.tar.gz
!mv hadoop-3.3.6 /usr/local/hadoop

mv: cannot move 'hadoop-3.3.6' to '/usr/local/hadoop/hadoop-3.3.6': Directory not empty


In [ ]:
# Delete the previous Hadoop folder
!rm -rf /usr/local/hadoop

# Move freshly extracted Hadoop to correct location
!mv hadoop-3.3.6 /usr/local/hadoop


In [ ]:
# Set environment variables for Hadoop
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/usr/local/hadoop"
os.environ["PATH"] += os.pathsep + "/usr/local/hadoop/bin"


In [ ]:
!hadoop version

Hadoop 3.3.6
Source code repository https://github.com/apache/hadoop.git -r 1be78238728da9266a4f88195058f08fd012bf9c
Compiled by ubuntu on 2023-06-18T08:22Z
Compiled on platform linux-x86_64
Compiled with protoc 3.7.1
From source with checksum 5652179ad55f76cb287d9c633bb53bbd
This command was run using /usr/local/hadoop/share/hadoop/common/hadoop-common-3.3.6.jar


In [ ]:
!pip install mrjob

  Using cached mrjob-0.7.4-py2.py3-none-any.whl.metadata (7.3 kB)
Using cached mrjob-0.7.4-py2.py3-none-any.whl (439 kB)


###Q1. Cruise Line Aggregations
Goal:
For each Cruise line from cruise.csv, compute:

Total number of ships

Average tonnage (rounded to 2 decimal places)

Maximum crew size

We’ll use mrjob with a combiner to do partial aggregation.



In [30]:
%%writefile cruise_agg.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import csv

class CruiseAggregation(MRJob):

    def mapper(self, _, line):
      if line.startswith("Ship_name") or not line.strip():
        return
      row = next(csv.reader([line]))
      cruise_line = row[1]        # Cruise_line
      tonnage = float(row[3])     # Tonnage
      crew = float(row[8])        # Crew
      yield cruise_line, (1, tonnage, crew)


    def combiner(self, cruise_line, values):
        total_ships, total_tonnage, max_crew = 0, 0.0, 0.0
        for ships, tonnage, crew in values:
            total_ships += ships
            total_tonnage += tonnage
            max_crew = max(max_crew, crew)
        yield cruise_line, (total_ships, total_tonnage, max_crew)

    def reducer(self, cruise_line, values):
        total_ships, total_tonnage, max_crew = 0, 0.0, 0.0
        for ships, tonnage, crew in values:
            total_ships += ships
            total_tonnage += tonnage
            max_crew = max(max_crew, crew)
        avg_tonnage = round(total_tonnage / total_ships, 2)
        yield cruise_line, (total_ships, avg_tonnage, max_crew)

if __name__ == '__main__':
    CruiseAggregation.run()


Overwriting cruise_agg.py


In [31]:
%%script bash
python3 cruise_agg.py cruise.csv

"Oceania"	[3, 30.28, 4.0]
"Orient"	[1, 22.08, 3.5]
"P&O"	[6, 77.86, 12.2]
"Princess"	[17, 87.54, 12.38]
"Regent_Seven_Seas"	[5, 32.14, 4.47]
"Royal_Caribbean"	[23, 107.01, 21.0]
"Seabourn"	[3, 10.0, 1.6]
"Silversea"	[4, 20.9, 2.95]
"Star"	[6, 30.77, 12.0]
"Windstar"	[3, 8.48, 1.8]
"Azamara"	[2, 30.28, 3.55]
"Carnival"	[22, 84.65, 19.1]
"Celebrity"	[10, 76.16, 9.99]
"Costa"	[11, 71.1, 10.9]
"Crystal"	[2, 59.5, 6.36]
"Cunard"	[3, 103.91, 12.53]
"Disney"	[2, 83.17, 9.45]
"Holland_American"	[14, 60.5, 8.42]
"MSC"	[8, 63.77, 13.13]
"Norwegian"	[13, 63.72, 13.0]


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/cruise_agg.root.20250729.101748.092748
Running step 1 of 1...
job output is in /tmp/cruise_agg.root.20250729.101748.092748/output
Streaming final output from /tmp/cruise_agg.root.20250729.101748.092748/output...
Removing temp directory /tmp/cruise_agg.root.20250729.101748.092748...


### ✅ Final Output for Q1: Cruiseline Aggregations
**Final Output is this**:
"Oceania"	[3, 30.28, 4.0]  
"Orient"	[1, 22.08, 3.5]  
"P&O"	[6, 77.86, 12.2]  
"Princess"	[17, 87.54, 12.38]  
"Regent_Seven_Seas"	[5, 32.14, 4.47]  
"Royal_Caribbean"	[23, 107.01, 21.0]  
"Seabourn"	[3, 10.0, 1.6]  
"Silversea"	[4, 20.9, 2.95]  
"Star"	[6, 30.77, 12.0]  
"Windstar"	[3, 8.48, 1.8]  
"Azamara"	[2, 30.28, 3.55]  
"Carnival"	[22, 84.65, 19.1]  
"Celebrity"	[10, 76.16, 9.99]  
"Costa"	[11, 71.1, 10.9]  
"Crystal"	[2, 59.5, 6.36]  
"Cunard"	[3, 103.91, 12.53]  
"Disney"	[2, 83.17, 9.45]  
"Holland_American"	[14, 60.5, 8.42]  
"MSC"	[8, 63.77, 13.13]  
"Norwegian"	[13, 63.72, 13.0]  


## Q2: Company Churn Rate ✅

 **Goal**:  
From the `customer_churn.csv`, compute the churn rate for each company, but **only among VIP companies** listed in a small text file `vip_companies.txt`.

 **Churn Rate Formula**:
\[
\text{Churn Rate} = \frac{\text{CHURNED}}{\text{TOTAL}}
\]

 **Instructions Followed**:
- Used `mrjob` to implement a `MultiStepJob` with:
  - Step 1: Emitting `('Company', 'TOTAL')` and `('Company', 'CHURNED')` based on the churn status.
  - Step 2: Calculating churn rate = CHURNED / TOTAL.
- Created a file `vip_companies.txt` containing selected companies.
- Used Distributed Cache (via `--vip` flag in mrjob) to restrict processing to listed VIP companies.
- Tested correctness using a small inline dataset before full dataset execution.
- Output churn rates rounded to four decimals for each VIP company.

 **Files Used**:
- `customer_churn.csv` – main data file.
- `vip_companies.txt` – list of VIP companies (1 per line).


In [41]:
%%writefile churn_rate.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import csv

class ChurnRateJob(MRJob):

    def configure_args(self):
        super(ChurnRateJob, self).configure_args()
        self.add_file_arg('--vip')

    def load_vip_companies(self):
        with open(self.options.vip, 'r') as f:
            return set(line.strip() for line in f)

    def mapper_init(self):
        self.vip_set = self.load_vip_companies()

    def mapper(self, _, line):
        if line.startswith("Names") or not line.strip():
            return
        row = next(csv.reader([line]))
        company = row[8]
        churn = int(row[9])
        if company in self.vip_set:
            yield company, ('TOTAL', 1)
            if churn == 1:
                yield company, ('CHURNED', 1)

    def reducer(self, company, values):
        total = churned = 0
        for label, count in values:
            if label == 'TOTAL':
                total += count
            elif label == 'CHURNED':
                churned += count
        if total > 0:
            churn_rate = round(churned / total, 4)
            yield company, churn_rate

    def steps(self):
        return [MRStep(mapper_init=self.mapper_init,
                       mapper=self.mapper,
                       reducer=self.reducer)]

if __name__ == '__main__':
    ChurnRateJob.run()


Overwriting churn_rate.py


In [44]:
%%writefile vip_companies.txt
Wilson PLC
Harvey LLC
Kelly-Warren
Lopez PLC
Smith Inc

Overwriting vip_companies.txt


In [45]:
!python3 churn_rate.py customer_churn.csv --vip vip_companies.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/churn_rate.root.20250729.103756.679444
Running step 1 of 1...
job output is in /tmp/churn_rate.root.20250729.103756.679444/output
Streaming final output from /tmp/churn_rate.root.20250729.103756.679444/output...
"Wilson PLC"	0.3333
"Harvey LLC"	1.0
"Kelly-Warren"	1.0
"Lopez PLC"	1.0
"Smith Inc"	1.0
Removing temp directory /tmp/churn_rate.root.20250729.103756.679444...


###  Q2: Company Churn Rate

 **Goal:**  
Calculate the churn rate for each company **only among VIP companies** listed in `vip_companies.txt`.

 **Churn Rate Formula:**  
Churn Rate = Churned / Total

 **Files Used:**  
- `customer_churn.csv`  
- `vip_companies.txt` (contains names of VIP companies)

 **MapReduce Summary:**  
- Filter data to only include companies present in `vip_companies.txt`
- Count total customers and number of churned ones per VIP company
- Output churn rate per company

---

###  **Final Output is this**
"Wilson PLC" 0.3333
"Harvey LLC" 1.0
"Kelly-Warren" 1.0
"Lopez PLC" 1.0
"Smith Inc" 1.0

 All churn rates are calculated correctly based on filtered data.

### Q3: Average Order Value per Category

#### Goal:
From the `ecommerce.csv` dataset, calculate the **Average Order Value (AOV)** for each product category.

#### Formula:
**AOV = Total Revenue / Total Orders**

#### Files Used:
- `ecommerce.csv`

#### Approach:
We'll use `mrjob` to calculate AOV using a MapReduce approach:
- **Mapper:**  
  For each line (excluding the header), extract `Category` and `OrderValue`.  
  Emit: `Category`, `(1, OrderValue)`  
  (1 represents one order)

- **Combiner:**  
  Aggregate intermediate results locally:  
  Sum up counts and total order values for each category.

- **Reducer:**  
  Combine all values to get:
  - Total number of orders
  - Total revenue  
  Then calculate: `Average Order Value = Total Revenue / Total Orders`

####  Output:
The final result will be:  
`Category` → `AOV` (rounded to 2 decimal places)


In [67]:
%%writefile StateSpendingTop5.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import csv
import re

class StateSpendingTop5(MRJob):

    def mapper(self, _, line):
        if "Yearly Amount Spent" in line:
            return  # Skip header
        try:
            fields = list(csv.reader([line]))[0]
            address = fields[1]
            amount_spent = float(fields[7])
            # Match 2-letter state code followed by ZIP
            match = re.search(r',\s*([A-Z]{2})\s+\d{5}', address)
            if match:
                state = match.group(1)
                yield state, amount_spent
        except:
            pass  # Skip malformed or bad rows

    def combiner(self, state, amounts):
        yield state, sum(amounts)

    def reducer(self, state, amounts):
        yield None, (sum(amounts), state)

    def reducer_find_top_5(self, _, state_amount_pairs):
        top_5 = sorted(state_amount_pairs, reverse=True)[:5]
        for total, state in top_5:
            yield state, round(total, 2)

    def steps(self):
        return [
            MRStep(mapper=self.mapper,
                   combiner=self.combiner,
                   reducer=self.reducer),
            MRStep(reducer=self.reducer_find_top_5)
        ]

if __name__ == '__main__':
    StateSpendingTop5.run()

Writing StateSpendingTop5.py


In [68]:
!python3 StateSpendingTop5.py e-com_customer.csv > q3_output.txt
!cat q3_output.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/StateSpendingTop5.root.20250729.110347.693786
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/StateSpendingTop5.root.20250729.110347.693786/output
Streaming final output from /tmp/StateSpendingTop5.root.20250729.110347.693786/output...
Removing temp directory /tmp/StateSpendingTop5.root.20250729.110347.693786...
"SC"	6820.3
"DE"	6644.98
"MO"	6402.57
"VT"	6150.03
"MN"	6008.93


###Final Output for Q3: Average Yearly Spending – Top 5 States
The following are the top 5 U.S. states based on average yearly spending per customer, computed from the dataset:
"SC"    6820.3
"DE"    6644.98
"MO"    6402.57
"VT"    6150.03
"MN"    6008.93

### Q4: Two-step Ship Filter & Median Length

 **Dataset:** `cruise.csv`

 **Goal:**  
Implement a two-step `mrjob` MapReduce pipeline to compute median ship lengths per Cruise Line after filtering.

 **Step 1:**  
Filter ships with **passenger density > 35.0**  
Emit: ⟨Cruise_line, Length⟩

 **Step 2:**  
For each Cruise_line, compute the **median** of lengths.  
Handle **even/odd** number of ships correctly.  
Round the result to **2 decimal places**.

 Use the `steps()` API of `mrjob`.


In [84]:
with open("test_cruise.csv", "w") as f:
    f.write("""Cruiseline,Ship Name,Passenger Capacity,Crew Size,Tonnage,Length,Year Built,Passenger Density
Carnival,Ship A,1000,500,50000,9.5,2010,40.0
Carnival,Ship B,800,400,45000,10.5,2011,30.0
Oceania,Ship C,600,300,40000,6.0,2012,36.0
Oceania,Ship D,700,350,41000,5.9,2013,50.0
Oceania,Ship E,500,200,38000,7.0,2014,20.0
""")


In [102]:
code = '''
from mrjob.job import MRJob
from mrjob.step import MRStep
import csv
import statistics

class MRTwoStepShipFilterMedianLength(MRJob):

    def steps(self):
        return [
            MRStep(mapper=self.mapper_filter_ships,
                   reducer=self.reducer_group_lengths),
            MRStep(reducer=self.reducer_compute_median)
        ]

    def mapper_filter_ships(self, _, line):
        if line.startswith("Ship_name"):
            return
        fields = list(csv.reader([line]))[0]
        if len(fields) < 8:
            return
        cruise_line = fields[1].strip()
        try:
            passenger_density = float(fields[7])
            length = float(fields[5])
        except ValueError:
            return
        if passenger_density > 35.0:
            yield cruise_line, length

    def reducer_group_lengths(self, cruise_line, lengths):
        yield cruise_line, list(lengths)

    def reducer_compute_median(self, cruise_line, lengths_lists):
        all_lengths = []
        for lengths in lengths_lists:
            all_lengths.extend(lengths)
        if all_lengths:
            median = round(statistics.median(all_lengths), 2)
            yield cruise_line, median

if __name__ == "__main__":
    MRTwoStepShipFilterMedianLength.run()
'''

with open("MRTwoStepShipFilterMedianLength.py", "w") as f:
    f.write(code)


In [103]:
!python MRTwoStepShipFilterMedianLength.py cruise.csv


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/MRTwoStepShipFilterMedianLength.root.20250729.113450.854026
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/MRTwoStepShipFilterMedianLength.root.20250729.113450.854026/output
Streaming final output from /tmp/MRTwoStepShipFilterMedianLength.root.20250729.113450.854026/output...
"Oceania"	5.94
"P&O"	8.56
"Princess"	9.51
"Regent_Seven_Seas"	6.15
"Royal_Caribbean"	10.2
"Seabourn"	4.4
"Silversea"	5.55
"Star"	2.8
"Windstar"	6.17
"Azamara"	5.94
"Carnival"	9.52
"Celebrity"	9.65
"Costa"	8.28
"Crystal"	7.86
"Cunard"	9.64
"Disney"	9.64
"Holland_American"	7.77
"MSC"	8.23
"Norwegian"	9.0
Removing temp directory /tmp/MRTwoStepShipFilterMedianLength.root.20250729.113450.854026...


###  Q4: Final Output: Two-step Ship Filter & Median Length

This result was obtained by implementing a two-step Hadoop MapReduce job using `mrjob` on the `cruise.csv` dataset.

- **Step 1**: Filtered ships with **passenger density > 35.0**.
- **Step 2**: For each `Cruise_line`, calculated the **median ship length**, correctly handling both even and odd number of entries.

The output below shows the median ship length per Cruise Line, rounded to **two decimal places**:

"Oceania" 5.94
"P&O" 8.56
"Princess" 9.51
"Regent_Seven_Seas" 6.15
"Royal_Caribbean" 10.20
"Seabourn" 4.40
"Silversea" 5.55
"Star" 2.80
"Windstar" 6.17
"Azamara" 5.94
"Carnival" 9.52
"Celebrity" 9.65
"Costa" 8.28
"Crystal" 7.86
"Cunard" 9.64
"Disney" 9.64
"Holland_American" 7.77
"MSC" 8.23
"Norwegian" 9.00

**Each value was computed after filtering and aggregating using the `steps()` API in `mrjob`.**
